In [1]:

import boto3
import time

# =============================================================================
# SECTION 1: Setup
# =============================================================================

# Get AWS account and region
session = boto3.Session()
region = session.region_name or 'us-east-1'
sts_client = boto3.client('sts', region_name=region)
account_id = sts_client.get_caller_identity()['Account']

print(f"AWS Account: {account_id}")
print(f"AWS Region: {region}")


AWS Account: 875423407011
AWS Region: us-west-2


In [2]:
# Image details
source_image = 'public.ecr.aws/deep-learning-containers/vllm:0.11.2-gpu-py312-cu129-ubuntu22.04-sagemaker-v1.2'
target_repo_name = 'vllm'
target_tag = '0.11.2-sagemaker-v1.2'
target_image = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{target_repo_name}:{target_tag}'

print(f"\nSource: {source_image}")
print(f"Target: {target_image}")



Source: public.ecr.aws/deep-learning-containers/vllm:0.11.2-gpu-py312-cu129-ubuntu22.04-sagemaker-v1.2
Target: 875423407011.dkr.ecr.us-west-2.amazonaws.com/vllm:0.11.2-sagemaker-v1.2


In [3]:
# =============================================================================
# SECTION 2: Create ECR Repository (if it doesn't exist)
# =============================================================================

ecr_client = boto3.client('ecr', region_name=region)

print(f"\nCreating ECR repository: {target_repo_name}")

try:
    response = ecr_client.create_repository(
        repositoryName=target_repo_name,
        imageScanningConfiguration={'scanOnPush': False},
        imageTagMutability='MUTABLE'
    )
    print(f"✓ Repository created: {response['repository']['repositoryUri']}")
except ecr_client.exceptions.RepositoryAlreadyExistsException:
    print(f"✓ Repository already exists: {target_repo_name}")
except Exception as e:
    print(f"Error creating repository: {e}")
    raise


Creating ECR repository: vllm
✓ Repository created: 875423407011.dkr.ecr.us-west-2.amazonaws.com/vllm


In [4]:
# =============================================================================
# SECTION 3: Copy Image Using Docker Commands
# =============================================================================

print("\n" + "="*70)
print("DOCKER COMMANDS TO COPY IMAGE")
print("="*70)
print("\nRun these commands in your terminal (not in Jupyter):\n")

print("# 1. Get ECR login token")
print(f"aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com")

print("\n# 2. Pull the public image")
print(f"docker pull {source_image}")

print("\n# 3. Tag the image for your private ECR")
print(f"docker tag {source_image} {target_image}")

print("\n# 4. Push to your private ECR")
print(f"docker push {target_image}")

print("\n" + "="*70)
print("\nAfter running these commands, update your Python code to use:")
print(f"container_image = '{target_image}'")
print("="*70)


DOCKER COMMANDS TO COPY IMAGE

Run these commands in your terminal (not in Jupyter):

# 1. Get ECR login token
aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 875423407011.dkr.ecr.us-west-2.amazonaws.com

# 2. Pull the public image
docker pull public.ecr.aws/deep-learning-containers/vllm:0.11.2-gpu-py312-cu129-ubuntu22.04-sagemaker-v1.2

# 3. Tag the image for your private ECR
docker tag public.ecr.aws/deep-learning-containers/vllm:0.11.2-gpu-py312-cu129-ubuntu22.04-sagemaker-v1.2 875423407011.dkr.ecr.us-west-2.amazonaws.com/vllm:0.11.2-sagemaker-v1.2

# 4. Push to your private ECR
docker push 875423407011.dkr.ecr.us-west-2.amazonaws.com/vllm:0.11.2-sagemaker-v1.2


After running these commands, update your Python code to use:
container_image = '875423407011.dkr.ecr.us-west-2.amazonaws.com/vllm:0.11.2-sagemaker-v1.2'


In [7]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [9]:
!docker pull {source_image}

0.11.2-gpu-py312-cu129-ubuntu22.04-sagemaker-v1.2: Pulling from deep-learning-containers/vllm

7c81b81a: Pulling fs layer 
1d4013a9: Pulling fs layer 
44d2921c: Pulling fs layer 
bcbddda3: Pulling fs layer 
f4fd0a07: Pulling fs layer 
fd421fac: Pulling fs layer 
56dea272: Pulling fs layer 
19e81d20: Pulling fs layer 
f2253cd0: Pulling fs layer 
79b2359a: Pulling fs layer 
10f4bfa0: Pulling fs layer 
f84485b3: Pulling fs layer 
9bff06dd: Pulling fs layer 
82d56f38: Pulling fs layer 
e34c04f6: Pulling fs layer 
c2a92587: Pulling fs layer 
63af10e1: Pulling fs layer 
e7a6b91d: Pulling fs layer 
e5fc94f4: Pulling fs layer 
4c99812a: Pulling fs layer 
b97da028: Pulling fs layer 
c54de256: Pulling fs layer 
647d6b16: Pulling fs layer 
4201d602: Pulling fs layer 
298be83f: Pulling fs layer 
a7ff3083: Pulling fs layer 
cb1e1a39: Pulling fs layer 
6e714056: Pulling fs layer 
6e9ab7e0: Pulling fs layer 
117692f9: Pulling fs layer 
2560f0b0: Pulling fs layer 
ce5849a2: Pulling fs layer 
7d5e212e:

In [10]:
!docker tag {source_image} {target_image}
!docker push {target_image}

The push refers to repository [875423407011.dkr.ecr.us-west-2.amazonaws.com/vllm]

1ca4b8f8: Preparing 
b5e08e59: Preparing 
c50b3475: Preparing 
a8d0980c: Preparing 
f8108fc5: Preparing 
479f11e9: Preparing 
336d4a30: Preparing 
d64eece1: Preparing 
e13b6bf0: Preparing 
6f021e3d: Preparing 
387e0af4: Preparing 
5f37a2d6: Preparing 
b1e18656: Preparing 
479f11e9: Waiting g 
d64eece1: Waiting g 
8309da02: Preparing 
e13b6bf0: Waiting g 
ed2feb83: Preparing 
5f37a2d6: Waiting g 
f021e3d: Waiting g 
b1e18656: Waiting g 
3ce083eb: Waiting g 
8309da02: Waiting g 
165f0350: Waiting g 
46c1b0b5: Waiting g 
eb0b0e04: Waiting g 
5c8b355d: Waiting g 
d3c0dfb9: Waiting g 
2a083b45: Preparing 
2a083b45: Waiting g 
210f8ac0: Waiting g 
bdeeea2f: Preparing 
cc501b40: Preparing 
56be1356: Waiting g 
30fa613a: Preparing 
07956969: Preparing 
30fa613a: Waiting g 
bdeeea2f: Waiting g 
07956969: Waiting g 
94e75ffe: Preparing 
c8b355d: Pushed   10.68GB/10.65GBPushing  59.67MB/348.3MBPushing  66.74MB/589.